In [ ]:
from google.colab import files
import os

# 1. Upload the kaggle.json you just downloaded
files.upload()

# 2. Setup the directory for the API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# 1. Download the CIFAKE dataset
!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images

# 2. Unzip it (it will create a folder called 'train' and 'test')
import zipfile
with zipfile.ZipFile("cifake-real-and-ai-generated-synthetic-images.zip", 'r') as zip_ref:
    zip_ref.extractall("cifake_data")

# 3. Define your paths for Member 1 code
TRAIN_DIR = "/content/cifake_data/train"
TEST_DIR = "/content/cifake_data/test"

Dataset URL: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images
License(s): other
  0% 0.00/105M [00:00<?, ?B/s]
100% 105M/105M [00:00<00:00, 1.29GB/s]


In [ ]:
!pip install -q tqdm torch torchvision grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import cv2

# --- Preprocessing Utilities ---
def high_pass_filter(image_tensor):
    img = image_tensor.permute(1, 2, 0).numpy()
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    hpf = img - blurred
    return torch.from_numpy(hpf).permute(2, 0, 1)

def simulate_jpeg_compression(image_pil, quality=10):
    import io
    from PIL import Image
    buffer = io.BytesIO()
    image_pil.save(buffer, "JPEG", quality=quality)
    buffer.seek(0)
    return Image.open(buffer)

def convert_to_ycbcr(image_pil):
    return image_pil.convert('YCbCr')

# --- Data Pipeline Setup ---
stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

train_transform = transforms.Compose([
    # Uncomment for Unique Flexes:
    transforms.Lambda(lambda x: simulate_jpeg_compression(x, quality=30)),
    # transforms.Lambda(lambda x: convert_to_ycbcr(x)),
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: high_pass_filter(x)),
    transforms.Normalize(*stats)
])

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# --- Model Definition ---
def get_detector_model():
    model = models.resnet18(weights='IMAGENET1K_V1')
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    return model

# --- Training Logic ---
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss/len(loader), 100.*correct/total

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100.*correct/total

In [ ]:
# 1. Initialize Loaders
train_set = torchvision.datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_set = torchvision.datasets.ImageFolder(root=TEST_DIR, transform=test_transform)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# 2. Initialize Hardware & Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_detector_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Main Loop
num_epochs = 10
print(f"Training on {device}...")

for epoch in range(num_epochs):
    loss, acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_acc = evaluate(model, test_loader, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {loss:.4f} | Train Acc: {acc:.2f}% | Val Acc: {val_acc:.2f}%")
    torch.save(model.state_dict(), 'cifake_resnet18_latest.pth')

print("Training Complete! Saved weights to 'cifake_resnet18_latest.pth'")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 184MB/s]


Training on cuda...


Training: 100%|██████████| 782/782 [01:08<00:00, 11.41it/s]


Epoch [1/10] | Loss: 0.2165 | Train Acc: 91.16% | Val Acc: 93.79%


...

Epoch [8/10] | Loss: 0.0961 | Train Acc: 96.28% | Val Acc: 95.14%


Training: 100%|██████████| 782/782 [01:06<00:00, 11.70it/s]


Epoch [9/10] | Loss: 0.0781 | Train Acc: 97.03% | Val Acc: 95.80%


Training: 100%|██████████| 782/782 [01:06<00:00, 11.82it/s]


Epoch [10/10] | Loss: 0.0605 | Train Acc: 97.76% | Val Acc: 96.19%
Training Complete! Saved weights to 'cifake_resnet18_latest.pth'
